In [14]:
import pandas as pd
import os
import gzip
from Bio import SeqIO, Entrez
from matplotlib import pyplot as plt
import numpy as np

# TODO: implement for other datasets

In [4]:
df = pd.read_csv('../data/dataset1/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('../data/dataset1/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

dfother = pd.DataFrame()
it = len(df)

# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'Tax ID']=df2.at[i, 'Tax ID']
    it+=1


df.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

mergedDf = pd.concat([df, dfother], axis=0)

In [5]:
print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9423   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9424   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9425  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9426   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9427    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [14]:
# split up into chunks so the ncbi genome download can handle it
for i in range(4):
    with open(f'ids-{i}.txt', 'w') as f:
        for index, row in mergedDf.iterrows():
            if (int(index) >= 2357*i and int(index) < 2357*(i+1)):
                f.write(row['ID']+'\n')

In [45]:
x = 0
for i in range(4):
    with open(f'ids-{i}.txt', 'rb') as f:
        for j in f.readlines():
            x+=1
print(x)

9428


In [44]:
#unzip each gz file in the refseq directory
num = 0
for folder in os.listdir('refseq/viral'):
    num+=1
    if folder != '.DS_Store':
        for file in os.listdir(f'refseq/viral/{folder}'):
            if file.endswith('.gz'):
                with gzip.open(f'refseq/viral/{folder}/{file}', 'rb') as f_in:
                    with open(f'dataset1/{file[:-3]}', 'wb') as f_out:
                        f_out.write(f_in.read())
                        num+=1
q
print(f'Unzipped {num} files')

Unzipped 13910 files


In [ ]:
# extract NCBI id from each file
for file in os.listdir('dataset1'):
    n = 0

    fasta_sequences = SeqIO.parse(open(f"dataset1/{file}"),'genbank')
    # for record in fasta_sequences:
    #     print(record.id)
    #     print(record.seq)
    
    fastas = [x for x in fasta_sequences]
    for fasta in fastas:
        if n == 0:
            mergedDf.loc[mergedDf['Tax ID'] ==]
            n+=1
        else:
            mergedDf.at[int(file[:-3]), 'NCBI ID'] = mergedDf.at[int(file[:-3]), 'NCBI ID'] + ',' + fasta.id
            n+=1

In [21]:
# baltimore classification
dnafamilies = set(['adenoviridae','papillomaviridae','papovaviridae', 
'parvoviridae', 'herpesviridae', 'poxviridae', 'anelloviridae', 'pleolipoviridae'])

rnafamilies = set(['reoviridae', 'picornaviridae', 
'caliciviridae', 'togaviridae', 'arenaviridae', 
'flaviviridae', 'orthomyxoviridae', 'paramyxoviridae', 'bunyaviridae', 
'rhabdoviridae', 'filoviridae', 'coronaviridae', 'astroviridae', 
'bornaviridae', 'arteriviridae', 'hepaciviridae'])


df = pd.DataFrame(columns=['family','accession', 'name','DNA Sequence'])
print(len(os.listdir('dataset1')))
commons = {}
for file in os.listdir('dataset1')[:40]:
    # print(file)
    n = 0

    # print(type(i))
    fasta_sequences = SeqIO.parse(open(f"dataset1/{file}"),'genbank')
    # for record in fasta_sequences:
    #     print(record.id)
    #     print(record.seq)
    
    fastas = [x for x in fasta_sequences]
    # print(fastas)
    for fasta in fastas:
        fasta.annotations['taxonomy']==None and print(fasta.annotations['taxonomy']==None)
        print(vars(fasta))
        accession, sequence, taxonomy, name = fasta.id, str(fasta.seq), fasta.annotations['taxonomy'], fasta.description
        family = None
        # print(taxonomy)
        for tax in taxonomy:
            commons[tax] = commons.get(tax, 0) + 1
            if tax.lower() in rnafamilies or tax.lower() in dnafamilies:
                family = tax
                # break
            
        if family == None:
            family = 'Other'

                
        df = pd.concat([pd.DataFrame({'family':family,'accession':accession, 'name':name.split(",")[0],'DNA Sequence':sequence}, index=[0]),df], ignore_index=True)
        
        # print(name)
        # print(sequence)
        n+=1
    # print(n)
    

6955
{'_seq': Seq('GTTCGCAGGCTATGCAGATTGCCGCTTTGCAGGTAGGTCAGTAATGTTTCTTGC...TGA'), 'id': 'NC_011356.1', 'name': 'NC_011356', 'description': 'Enterobacteria phage YYZ-2008, complete prophage genome', 'dbxrefs': ['BioProject:PRJNA485481'], 'annotations': {'molecule_type': 'DNA', 'topology': 'linear', 'data_file_division': 'PHG', 'date': '12-OCT-2021', 'accessions': ['NC_011356'], 'sequence_version': 1, 'keywords': ['RefSeq'], 'source': 'Enterobacteria phage YYZ-2008', 'organism': 'Enterobacteria phage YYZ-2008', 'taxonomy': ['Viruses', 'Duplodnaviria', 'Heunggongvirae', 'Uroviricota', 'Caudoviricetes', 'Pankowvirus', 'Pankowvirus YYZ2008'], 'references': [Reference(title='Direct Submission', ...), Reference(title='Direct Submission', ...)], 'comment': 'PROVISIONAL REFSEQ: This record has not yet been subject to final\nNCBI review. The reference sequence was derived from FJ184280.\nCOMPLETENESS: full length.'}, '_per_letter_annotations': {}, 'features': [SeqFeature(FeatureLocation(ExactPo

In [13]:
print(df['family'].value_counts())
# print(commons)
# print(dict(sorted(commons.items(), key=lambda item: item[1], reverse=True)))


Other               7664
Rhabdoviridae        171
Papillomaviridae     164
Flaviviridae         132
Picornaviridae       119
Orthomyxoviridae     105
Parvoviridae         103
Adenoviridae          88
Herpesviridae         79
Anelloviridae         67
Arenaviridae          64
Coronaviridae         59
Poxviridae            51
Astroviridae          46
Caliciviridae         41
Paramyxoviridae       40
Togaviridae           23
Arteriviridae         18
Bornaviridae          14
Pleolipoviridae        8
Filoviridae            8
Name: family, dtype: int64
